In [ ]:
import pandas as pd
import re
import jellyfish
from unidecode import unidecode
from rapidfuzz import fuzz, distance
from datasketch import MinHash, MinHashLSH
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import precision_recall_fscore_support
from joblib import Parallel, delayed

### 2. 数据预处理

在这一部分，我们对数据进行标准化处理：
- 将所有的名字转为小写字母
- 去除名字中的音符符号（如重音）
- 清理特殊字符，只保留字母、数字和空格
- 合并多余的空格

这一步确保我们对数据进行统一标准化，以便后续匹配。

In [ ]:
# 处理名称的标准化方法
def normalize(name: str) -> str:
    # 去除音符符号
    name = unidecode(name)
    # 转为小写
    name = name.lower()
    # 清理特殊字符（除字母、数字、空格外的所有字符）
    name = re.sub(r"[^a-z0-9\s]", " ", name)
    # 合并多余的空格
    name = " ".join(name.split())
    return name

### 3. LSH (MinHash) 特征计算

通过计算每个名称的 MinHash 特征来准备阻塞步骤。MinHash 是一种用于**局部敏感哈希 (LSH)** 的技术，可以通过比较小的指纹来加速相似性比较。


In [ ]:
# LSH (MinHash) 特征计算
def minhash_features(name):
    m = MinHash()
    for d in name.split():
        m.update(d.encode('utf8'))
    return m


### 4. 阻塞技术：使用 LSH 或 Soundex

我们在这里实现了两种阻塞方法：
1. **LSH**（局部敏感哈希）：我们通过 MinHash 计算每个名称的指纹，并使用 LSH 来加速记录匹配。
2. **Soundex**：基于名称的 Soundex 码进行阻塞，适用于拼写变体较小的情况。

可以通过设置 `use_lsh` 为 `True` 或 `False` 来选择不同的阻塞方法。


In [ ]:
# 阻塞操作 - 使用LSH
def generate_block_keys(df, use_lsh=False):
    if use_lsh:
        df['minhash'] = df['norm_name'].apply(minhash_features)
        return df
    else:
        df["soundex_key"] = df["norm_name"].apply(lambda x: jellyfish.soundex(x))
        return df


### 5. LSH 阻塞实现

我们通过 LSH 将相似的记录分到相同的块中。这样，我们只需比较属于同一块的记录，避免了计算每个记录对的所有可能组合。


In [ ]:
# LSH 阻塞
def lsh_blocking(df, threshold=0.8):
    lsh = MinHashLSH(threshold=threshold, num_perm=128)
    candidates = []
    for idx, row in df.iterrows():
        lsh.insert(row['ID'], row['minhash'])
    
    for idx, row in df.iterrows():
        result = lsh.query(row['minhash'])
        for match_id in result:
            if match_id != row['ID']:
                candidates.append((row['ID'], match_id))
    return candidates


### 6. 提取候选对的相似度特征

我们使用多种方法计算记录对之间的相似度，如 **Jaro-Winkler 相似度**、**Levenshtein 距离** 和 **Token Set Ratio**。这些特征将在后续的分类模型中作为输入。


In [ ]:
# 提取相似度特征
def build_features(a, b):
    return {
        "jw": fuzz.WRatio(a, b, score_cutoff=0)/100,  # Jaro-Winkler 相似度
        "lev_ratio": 1 - distance.Levenshtein.normalized_distance(a, b),  # Levenshtein 相似度
        "token_set": fuzz.token_set_ratio(a, b)/100,  # Token Set Ratio
        "prefix_match": int(a[:4] == b[:4]),  # 前缀匹配
        "len_diff": abs(len(a) - len(b)),  # 长度差异
        "same_soundex": int(jellyfish.soundex(a) == jellyfish.soundex(b))  # Soundex匹配
    }


### 7. 抽取候选对

我们从 `primary_df` 和 `alternate_df` 中抽取候选对。如果两个记录属于同一块（通过阻塞键判断），我们将计算它们之间的相似度特征。


In [ ]:
# 抽取候选对：同block的记录进行匹配
def extract_candidates(primary_df, alternate_df, use_lsh=False):
    candidates = []
    for _, primary_row in primary_df.iterrows():
        for _, alternate_row in alternate_df.iterrows():
            if (use_lsh and primary_row['minhash'] == alternate_row['minhash']) or \
               (not use_lsh and primary_row['soundex_key'] == alternate_row['soundex_key']):
                features = build_features(primary_row['norm_name'], alternate_row['norm_name'])
                candidates.append({
                    'id_left': primary_row['ID'],
                    'id_right': alternate_row['ID'],
                    'name_left': primary_row['NAME'],
                    'name_right': alternate_row['NAME'],
                    'Y/N': 'Y' if primary_row['ID'] == alternate_row['ID'] else 'N',
                    **features
                })
    return pd.DataFrame(candidates)


### 8. 读取数据

在这一部分，我们加载了 `primary.csv` 和 `alternate.csv` 数据，并进行标准化处理。同时，我们加载 `test_01.csv` 数据，准备进行映射任务。


In [ ]:
# 读取数据
primary_df = pd.read_csv('/mnt/data/primary.csv')
alternate_df = pd.read_csv('/mnt/data/alternate.csv')
test_df = pd.read_csv('/mnt/data/test_01.csv')

# 标准化数据
primary_df['norm_name'] = primary_df['NAME'].apply(normalize)
alternate_df['norm_name'] = alternate_df['NAME'].apply(normalize)

# 生成阻塞键
use_lsh = True  # 使用 LSH 阻塞
primary_df = generate_block_keys(primary_df, use_lsh=use_lsh)
alternate_df = generate_block_keys(alternate_df, use_lsh=use_lsh)


### 9. 训练模型：随机森林和朴素贝叶斯

我们使用 **随机森林（RandomForestClassifier）** 和 **朴素贝叶斯（Naive Bayes）** 来对记录匹配进行分类。我们评估了两个模型的 **Precision**、**Recall** 和 **F1-Score**，以比较它们的表现。


In [ ]:
# 随机森林模型
rf_model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
rf_model.fit(X_train, y_train)

# 评估：使用5折交叉验证
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cross_val_score(rf_model, X, y, cv=cv, scoring='accuracy').mean()

# 计算模型在测试集上的表现
y_pred_rf = rf_model.predict(X_test)
precision_rf, recall_rf, f1_rf, _ = precision_recall_fscore_support(y_test, y_pred_rf, average='binary')

print(f"Random Forest Precision: {precision_rf:.3f}, Recall: {recall_rf:.3f}, F1-Score: {f1_rf:.3f}")

# 朴素贝叶斯模型
nb_model = BernoulliNB()
nb_model.fit(X_train, y_train)

# 评估贝叶斯模型
y_pred_nb = nb_model.predict(X_test)
precision_nb, recall_nb, f1_nb, _ = precision_recall_fscore_support(y_test, y_pred_nb, average='binary')

print(f"Naive Bayes Precision: {precision_nb:.3f}, Recall: {recall_nb:.3f}, F1-Score: {f1_nb:.3f}")


### 10. 预测结果输出

在此部分，我们输出了每个测试样本的预测结果以及相应的概率。最终结果被保存到 `predictions_with_proba.csv` 文件中，供后续分析使用。


In [ ]:
# 输出模型概率预测 (proba)
rf_probs = rf_model.predict_proba(X_test)[:, 1]  # 获取 positive 类别的概率
nb_probs = nb_model.predict_proba(X_test)[:, 1]

# 输出结果 (Y/N) 及对应的概率
output_df = X_test.copy()
output_df['rf_y_pred'] = rf_model.predict(X_test)
output_df['rf_y_proba'] = rf_probs
output_df['nb_y_pred'] = nb_model.predict(X_test)
output_df['nb_y_proba'] = nb_probs

# 合并原始名称数据
output_df['id_left'] = y_test.index
output_df['id_right'] = X_test.index

# 最终结果导出
output_df.to_csv('predictions_with_proba.csv', index=False)
